<a href="https://colab.research.google.com/github/awaiskhan005/DATA-SCIENCE-AND-AI-/blob/main/Frankliin_project_With_MOCK_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')
# Import Required Libraries
import os
import joblib
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Install the 'ta' library if it's not already installed
!pip install ta

# Install the 'python-binance' library if it's not already installed
!pip install python-binance # This line is added to install the missing library

# Now import from 'ta'
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands
from ta.trend import EMAIndicator
from binance.client import Client # This import should now work
from datetime import datetime, timedelta, timezone
import time

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=ef4394aa57d2c386b00465f60ab55486cd8f7d10dbf37958b56eccdf1a035ff8
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 53.0 MB/s eta 0:00:00


In [2]:
import os
import joblib
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands
from ta.trend import EMAIndicator
from binance.client import Client
from datetime import datetime, timedelta, timezone
import time
# Import Required Libraries
import os
import joblib
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands
from ta.trend import EMAIndicator
from datetime import datetime, timedelta, timezone
import time
from unittest.mock import MagicMock


In [3]:
# Mock Binance Client
class MockBinanceClient:
    def get_historical_klines(self, symbol, interval, lookback):
        # Generate mock data for 500 candles
        timestamps = pd.date_range(start="2020-01-01", periods=500, freq="6H")
        data = {
            'timestamp': timestamps,
            'open': np.random.uniform(100, 200, len(timestamps)),
            'high': np.random.uniform(200, 300, len(timestamps)),
            'low': np.random.uniform(50, 100, len(timestamps)),
            'close': np.random.uniform(100, 200, len(timestamps)),
            'volume': np.random.uniform(1000, 5000, len(timestamps)),
            'close_time': timestamps,
            'quote_asset_volume': np.random.uniform(500, 1500, len(timestamps)),
            'number_of_trades': np.random.randint(10, 100, len(timestamps)),
            'taker_buy_base_asset_volume': np.random.uniform(100, 500, len(timestamps)),
            'taker_buy_quote_asset_volume': np.random.uniform(100, 500, len(timestamps)),
            'ignore': [0] * len(timestamps),
        }
        df = pd.DataFrame(data)
        return df.values.tolist()


In [4]:
# Replace Binance Client with Mock Client
client = MockBinanceClient()

In [5]:
# Define Directories and Symbols
model_dir = "models"
os.makedirs(model_dir, exist_ok=True)

symbols = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "SOLUSDT"]

In [6]:
# Fetch and Process Data
def fetch_candles(symbol, interval, lookback):
    candles = client.get_historical_klines(symbol, interval, lookback)
    df = pd.DataFrame(candles, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', errors='coerce')
    df.set_index('timestamp', inplace=True)

    for col in ['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

In [7]:
# Fetch and Process Data
def fetch_candles(symbol, interval, lookback):
    candles = client.get_historical_klines(symbol, interval, lookback)
    df = pd.DataFrame(candles, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', errors='coerce')
    df.set_index('timestamp', inplace=True)

    for col in ['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

In [8]:
# Calculate Indicators
def calculate_indicators(df):
    bb = BollingerBands(df['close'], window=12, window_dev=2)
    df['bb_low'] = bb.bollinger_lband()
    df['bb_high'] = bb.bollinger_hband()
    df['rsi'] = RSIIndicator(df['close'], window=14).rsi()
    stoch_rsi = (df['rsi'] - df['rsi'].rolling(window=14).min()) / (df['rsi'].rolling(window=14).max() - df['rsi'].rolling(window=14).min())
    df['stoch_rsi'] = stoch_rsi
    df['fast_ema'] = EMAIndicator(close=df['close'], window=5).ema_indicator()
    df['slow_ema'] = EMAIndicator(close=df['close'], window=15).ema_indicator()
    df['sma_30'] = df['close'].rolling(window=30).mean()
    return df


In [9]:
# Train Models for Each Symbol
for symbol in symbols:
    df = fetch_candles(symbol, "6h", '500 days ago UTC')
    df = calculate_indicators(df)
    df['future_return'] = df['close'].shift(-80) / df['close'] - 1
    df['target'] = (df['future_return'] > 0.6).astype(int)
    df.dropna(inplace=True)

In [10]:

    # Split Data into Train/Test
    features = df.drop(columns=['target'])
    target = df['target']
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [13]:
# Train Models for Each Symbol
for symbol in symbols:
    df = fetch_candles(symbol, "6h", '500 days ago UTC')
    df = calculate_indicators(df)
    df['future_return'] = df['close'].shift(-80) / df['close'] - 1
    df['target'] = (df['future_return'] > 0.6).astype(int)
    df.dropna(inplace=True)

    # Split Data into Train/Test
    features = df.drop(columns=['target', 'close_time']) # Drop 'close_time' column
    target = df['target']
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # Train XGBoost Model
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)

    # Save Model
    model_path = os.path.join(model_dir, f"{symbol}_xgboost_model.pkl")
    joblib.dump(model, model_path)

    # Evaluate Model
    y_pred = model.predict(X_test)
    print(f"Model for {symbol}:")
    print(classification_report(y_test, y_pred))

Model for BTCUSDT:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        77
           1       1.00      1.00      1.00         2

    accuracy                           1.00        79
   macro avg       1.00      1.00      1.00        79
weighted avg       1.00      1.00      1.00        79

Model for ETHUSDT:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        74
           1       1.00      1.00      1.00         5

    accuracy                           1.00        79
   macro avg       1.00      1.00      1.00        79
weighted avg       1.00      1.00      1.00        79

Model for BNBUSDT:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        75
           1       1.00      1.00      1.00         4

    accuracy                           1.00        79
   macro avg       1.00      1.00      1.00        79
weighted avg     

In [14]:

    # Save Model
    model_path = os.path.join(model_dir, f"{symbol}_xgboost_model.pkl")
    joblib.dump(model, model_path)

    # Evaluate Model
    y_pred = model.predict(X_test)
    print(f"Model for {symbol}:")
    print(classification_report(y_test, y_pred))

Model for SOLUSDT:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99        75
           1       1.00      0.75      0.86         4

    accuracy                           0.99        79
   macro avg       0.99      0.88      0.93        79
weighted avg       0.99      0.99      0.99        79



In [29]:
# Import Required Libraries
import os
import joblib
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands
from ta.trend import EMAIndicator
from datetime import datetime, timedelta, timezone
import time
from unittest.mock import MagicMock

# Mock Binance Client
class MockBinanceClient:
    def get_historical_klines(self, symbol, interval, lookback):
        # Generate mock data for 500 candles
        timestamps = pd.date_range(start="2020-01-01", periods=500, freq="6H")
        data = {
            'timestamp': timestamps,
            'open': np.random.uniform(100, 200, len(timestamps)),
            'high': np.random.uniform(200, 300, len(timestamps)),
            'low': np.random.uniform(50, 100, len(timestamps)),
            'close': np.random.uniform(100, 200, len(timestamps)),
            'volume': np.random.uniform(1000, 5000, len(timestamps)),
            'close_time': timestamps,
            'quote_asset_volume': np.random.uniform(500, 1500, len(timestamps)),
            'number_of_trades': np.random.randint(10, 100, len(timestamps)),
            'taker_buy_base_asset_volume': np.random.uniform(100, 500, len(timestamps)),
            'taker_buy_quote_asset_volume': np.random.uniform(100, 500, len(timestamps)),
            'ignore': [0] * len(timestamps),
        }
        df = pd.DataFrame(data)
        return df.values.tolist()

# Replace Binance Client with Mock Client
client = MockBinanceClient()

# Define Directories and Symbols
model_dir = "models"
os.makedirs(model_dir, exist_ok=True)

symbols = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "SOLUSDT"]

# Fetch and Process Data
def fetch_candles(symbol, interval, lookback):
    candles = client.get_historical_klines(symbol, interval, lookback)
    df = pd.DataFrame(candles, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', errors='coerce')
    df.set_index('timestamp', inplace=True)

    for col in ['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

# Calculate Indicators
def calculate_indicators(df):
    bb = BollingerBands(df['close'], window=12, window_dev=2)
    df['bb_low'] = bb.bollinger_lband()
    df['bb_high'] = bb.bollinger_hband()
    df['rsi'] = RSIIndicator(df['close'], window=14).rsi()
    stoch_rsi = (df['rsi'] - df['rsi'].rolling(window=14).min()) / (df['rsi'].rolling(window=14).max() - df['rsi'].rolling(window=14).min())
    df['stoch_rsi'] = stoch_rsi
    df['fast_ema'] = EMAIndicator(close=df['close'], window=5).ema_indicator()
    df['slow_ema'] = EMAIndicator(close=df['close'], window=15).ema_indicator()
    df['sma_30'] = df['close'].rolling(window=30).mean()
    return df

# Train Models for Each Symbol
for symbol in symbols:
    df = fetch_candles(symbol, "6h", '500 days ago UTC')
    df = calculate_indicators(df)
    df['future_return'] = df['close'].shift(-80) / df['close'] - 1
    df['target'] = (df['future_return'] > 0.6).astype(int)
    df.dropna(inplace=True)

    # Select only numeric features
    features = df.select_dtypes(include=[np.number]).drop(columns=['target', 'future_return'], errors='ignore')
    target = df['target']

    # Split Data into Train/Test
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # Train XGBoost Model
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)

    # Save Model
    model_path = os.path.join(model_dir, f"{symbol}_xgboost_model.pkl")
    joblib.dump(model, model_path)

    # Evaluate Model
    y_pred = model.predict(X_test)
    print(f"Model for {symbol}:")
    print(classification_report(y_test, y_pred))

# Use Trained Models to Predict Probabilities

def predict_with_trained_models():
    results = {}
    for symbol in symbols:
        model_path = os.path.join(model_dir, f"{symbol}_xgboost_model.pkl")
        if os.path.exists(model_path):
            model = joblib.load(model_path)
            df = fetch_candles(symbol, "6h", '80 candles ago UTC')
            df = calculate_indicators(df)
            features = df.select_dtypes(include=[np.number]).iloc[-1:].drop(columns=['target', 'future_return'], errors='ignore')
            probabilities = model.predict_proba(features)
            percentage_chances = probabilities[:, 1] * 100
            results[symbol] = percentage_chances[0]
    return results

results = predict_with_trained_models()
for symbol, prob in results.items():
    print(f"Probability of price increase >60% for {symbol}: {prob:.2f}%")

# Integration with Trading System

def execute_buy_order_with_model(client, symbol, usdt_to_use):
    results = predict_with_trained_models()
    probability = results.get(symbol, 0)
    if probability > 60:
        print(f"Executing buy order for {symbol} with {usdt_to_use} USDT. Probability: {probability:.2f}%")
        # Mocking buy order execution
        print(f"Buy order executed for {symbol}.")
    else:
        print(f"Buy signal not strong enough for {symbol}. Probability: {probability:.2f}%")


Model for BTCUSDT:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99        77
           1       0.50      0.50      0.50         2

    accuracy                           0.97        79
   macro avg       0.74      0.74      0.74        79
weighted avg       0.97      0.97      0.97        79

Model for ETHUSDT:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97        75
           1       0.00      0.00      0.00         4

    accuracy                           0.95        79
   macro avg       0.47      0.50      0.49        79
weighted avg       0.90      0.95      0.92        79

Model for BNBUSDT:
              precision    recall  f1-score   support

           0       0.96      0.95      0.95        75
           1       0.20      0.25      0.22         4

    accuracy                           0.91        79
   macro avg       0.58      0.60      0.59        79
weighted avg     